In [ ]:
#!g1.1
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import os
import shutil
from PIL import Image
import pandas as pd
import numpy as np
import math
import wandb
import sys
from torchvision.transforms.functional import InterpolationMode

In [ ]:
#!g1.1
from MyDataset import MyDataset
from train import train_m
from predict import predict
from loader import get_data

In [46]:
#!g1.1
transform_sem = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform_sem = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

df = pd.read_csv('data/bhw1-dataset/labels.csv')
trainval = MyDataset(path="data/bhw1-dataset/trainval", df=df, load_to_ram=False, transform=transform_sem) 
test = MyDataset(path="data/bhw1-dataset/test", df=None, load_to_ram=False, transform=test_transform_sem) 
cnt_l = len(np.unique(np.array(trainval.labels)))
train, val = random_split(trainval, [int(0.9*len(trainval)),len(trainval)- int(0.9*len(trainval))])

In [47]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [48]:
#!g1.1
from torchvision.models import mobilenet_v3_large
model2 = mobilenet_v3_large()
model2.classifier[3] = nn.Linear(in_features=1280, out_features=200)
model2.to(device)
print(device)

cuda:0


In [49]:
#!g1.1
wandb.init(project="project7", entity="arzhanandrey_team", settings=wandb.Settings(start_method="fork"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arzhanandrey (arzhanandrey_team). Use `wandb login --relogin` to force relogin


In [50]:
#!g1.1
from tqdm import tqdm 
train_loader, val_loader, test_loader = \
get_data(batch_size=64, data=train, shuffle=True), get_data(batch_size=64, data=val, shuffle=True),get_data(batch_size=64, data=test, shuffle=False)

optimizer = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

wandb.watch(model2)

train_loss_log, train_acc_log, val_loss_log, val_acc_log = train_m(device, model2, optimizer, 50, train_loader, val_loader, scheduler)
wandb.finish()

100%|██████████| 1407/1407 [03:25<00:00,  6.86it/s]


train_acc,▁▂▂▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇██████████
train_loss,█▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
val_loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train_acc,0.8816
train_loss,0.41732
val_acc,0.59315
val_loss,1.87623


In [51]:
#!g1.1
test_loader = get_data(batch_size=64, data=test, shuffle=False)
predicts = predict(device, model2, test_loader)в
df2 = pd.DataFrame(columns=['Id', 'Label'])
for i in range(len(predicts)):
    df2.loc[i, ['Id', 'Label']] = test.all_files[i], predicts[i].item()
print(df2.shape)

0
100
(10000, 2)


In [56]:
#!g1.1
df2.to_csv('mobile_plato_50ep.csv', index=False)

In [68]:
#!g1.1
wandb.init(project="project7", entity="arzhanandrey_team", settings=wandb.Settings(start_method="fork"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arzhanandrey (arzhanandrey_team). Use `wandb login --relogin` to force relogin


In [70]:
#!g1.1
model_f = torch.load('mobilenet_sem45.pt')
model2 = mobilenet_v3_large()
model2.classifier[3] = nn.Linear(in_features=1280, out_features=200)
model2.to(device)
print(device)
model2.load_state_dict(model_f)

cuda:0


<All keys matched successfully>

In [71]:
#!g1.1
df = pd.read_csv('data/bhw1-dataset/labels.csv')
trainval = MyDataset(path="data/bhw1-dataset/trainval", df=df, load_to_ram=False, transform=transform_sem) 
test = MyDataset(path="data/bhw1-dataset/test", df=None, load_to_ram=False, transform=test_transform_sem) 
cnt_l = len(np.unique(np.array(trainval.labels)))
train, val = random_split(trainval, [int(0.999*len(trainval)),len(trainval)- int(0.999*len(trainval))])

In [73]:
#!g1.1
wandb.init(project="project7", entity="arzhanandrey_team", settings=wandb.Settings(start_method="fork"))

In [74]:
#!g1.1
from tqdm import tqdm 
train_loader, val_loader, test_loader = \
get_data(batch_size=64, data=train, shuffle=True), get_data(batch_size=64, data=val, shuffle=True),get_data(batch_size=64, data=test, shuffle=False)

optimizer = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

wandb.watch(model2)

train_loss_log, train_acc_log, val_loss_log, val_acc_log = train_m(device, model2, optimizer, 10, train_loader, val_loader, scheduler)
wandb.finish()

100%|██████████| 1561/1561 [03:46<00:00,  6.90it/s]


train_acc,▁▁▂▂▂▃▃▃▇█
train_loss,█▇▇▇▆▆▆▆▂▁
val_acc,▇█▅▁▁▂▄▃▆▇
val_loss,▃▁▄█▇▇▅▇▂▃
train_acc,0.86522
train_loss,0.47648
val_acc,0.65017
val_loss,1.43172


In [75]:
#!g1.1
test = MyDataset(path="data/bhw1-dataset/test", df=None, load_to_ram=False, transform=test_transform_sem) 
test_loader = get_data(batch_size=64, data=test, shuffle=False)
predicts = predict(device, model2, test_loader)
df2 = pd.DataFrame(columns=['Id', 'Label'])
for i in range(len(predicts)):
    df2.loc[i, ['Id', 'Label']] = test.all_files[i], predicts[i].item()
print(df2.shape)
df2.to_csv('mobile_plato_sem55.csv', index=False)

0
100
(10000, 2)


In [76]:
#!g1.1
wandb.init(project="project7", entity="arzhanandrey_team", settings=wandb.Settings(start_method="fork"))

In [77]:
#!g1.1
from tqdm import tqdm 
train_loader, val_loader, test_loader = \
get_data(batch_size=64, data=train, shuffle=True), get_data(batch_size=64, data=val, shuffle=True),get_data(batch_size=64, data=test, shuffle=False)

optimizer = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

wandb.watch(model2)

train_loss_log, train_acc_log, val_loss_log, val_acc_log = train_m(device, model2, optimizer, 20, train_loader, val_loader, scheduler)
wandb.finish()

100%|██████████| 1561/1561 [03:43<00:00,  6.97it/s]


train_acc,▁▁▁▁▂▂▂▂▅▆▇▇▇▇██████
train_loss,████▇▇▇▇▄▃▂▂▂▂▁▁▁▁▁▁
val_acc,▃▅▄▂▄▃▄▁▆█▆▅▆▆▄▃▇▅▆▆
val_loss,▆▁▅▇▃█▆▇▂▃▃▅▄▄▄▅▄▄▁▁
train_acc,0.93182
train_loss,0.23818
val_acc,0.62413
val_loss,1.61549


In [79]:
#!g1.1
test = MyDataset(path="data/bhw1-dataset/test", df=None, load_to_ram=False, transform=test_transform_sem) 
test_loader = get_data(batch_size=64, data=test, shuffle=False)
predicts = predict(device, model2, test_loader)
df2 = pd.DataFrame(columns=['Id', 'Label'])
for i in range(len(predicts)):
    df2.loc[i, ['Id', 'Label']] = test.all_files[i], predicts[i].item()
print(df2.shape)
df2.to_csv('mobile_plato_sem65.csv', index=False)

0
100
(10000, 2)


In [ ]:
#!g1.1
